## Bundle Data

### 번들 내부

bundle=[img_0 depth_0 conf_0 info_0 img_1 depth_1 conf_1 info_1 ... img_n-1 depth_n-1 conf_n-1 imfo_n-1]


info={**'timestamp'**: 0.0, **'euler_angles'**: array([-0.6978792,  2.1804953, -1.5782039], dtype=float32), **'world_pose'**: array([[ 0.531039  , -0.5687025 ,  0.6281519 , -0.13826941],
       [-0.7661857 , -0.00567567,  0.64259416,  0.17472579],
       [-0.36187974, -0.82252353, -0.43874577, -0.20812593],
       [ 0.        ,  0.        ,  0.        ,  0.99999994]],
      dtype=float32), **'intrinsics'**: array([[1.6118971e+03, 0.0000000e+00, 9.5212964e+02],
       [0.0000000e+00, 1.6118971e+03, 7.1508081e+02],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]], dtype=float32), **'world_to_camera'**: array([[-0.5687026 , -0.00567572, -0.8225237 , -0.24883102],
       [-0.5310392 ,  0.76618576,  0.36187974, -0.13198233],
       [ 0.628152  ,  0.6425941 , -0.4387458 , -0.11673795],
       [ 0.        ,  0.        ,  0.        ,  0.99999994]],
      dtype=float32)}

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import math

In [2]:
#import 3D libraries
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import trimesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#현재 경로
print(os.getcwd())

\\172.26.39.200\data\sunjae\DSNeRF


In [4]:
def get_bundle_length(bundle):
    return len(bundle)//4

In [5]:
def show_img(*imgs):
    n=len(imgs)
    if n==1:
        plt.imshow(imgs[0])
        plt.show()
    elif n>5:
        fig, axs = plt.subplots(math.ceil(n/4), 4, figsize=(14, 8))
        for i,img in enumerate(imgs):
            axs[i//4,i%4].imshow(img)
            axs[i//4,i%4].set_title(f'image_{i}')
        plt.show()
    else:
        fig, axs = plt.subplots(1, n, figsize=(14, 8))
        for i,img in enumerate(imgs):
            axs[i].imshow(img)
            axs[i].set_title(f'image_{i}')
        plt.show()

In [6]:
#번들->포인트클라우드
#to visualize, make show flag True

def bundle2pt(bundle,frame,filename='',show_d=False,show_c=False,show_pc=False,allowed_conf=2,color_offset=[0,0,0],h=1):
    depth_image = bundle[f'depth_{frame}']
    rgb_image=bundle[f'img_{frame}']
    rgb_image=cv2.resize(rgb_image,dsize=(192,256))
    conf_image=bundle[f'conf_{frame}']
    if show_d:
        show_img(depth_image)
    if show_c:
        show_img(conf_image)
    M=bundle[f'info_{frame}'].item()['intrinsics']/7.5
    fx=M[0][0]
    fy=M[1][1]
    cx=M[0][2]
    cy=M[1][2]
    pcd = []
    colors=[]
    height, width = depth_image.shape
    height//=h
    for i in range(height):
        for j in range(width):
            if conf_image[i][j]<allowed_conf:
                continue
            #normalize coordinate
            z = depth_image[i][j]
            x = (j - cx) * z / fx 
            y = (i - cy) * z / fy
            pcd.append([x,y,z])
            colors.append((rgb_image[i][j]+color_offset)/255)
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
    # Visualize:
    if show_pc:
        o3d.visualization.draw_geometries([pcd_o3d])
    if filename:
        o3d.io.write_point_cloud(f'{filename}.ply', pcd_o3d, write_ascii=False, compressed=False, print_progress=False)
        print(f'save as {filename}.ply')
    
    return pcd_o3d

In [7]:
#save all images in bundle. 
#z is for step: save bundle 0,2,4,6,8, ...  then set z=2
#index_list is for specific indices: save bundle frame with 1,4,5,6,12,.. then set index_list=[1,4,5,6,12,..]

def save_bundle_imgs(bundle,directory,z=1,index_list=[]):
    n=(len(bundle)-1)//4
    if not os.path.exists(directory):
            os.makedirs(directory)
    if index_list:
        for i in index_list:
            img=bundle[f'img_{i}']
            if i%z==0:
                cv2.imwrite(f"{directory}/{i//z:0>3}.png",cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if i==n//2:
                print("...50% saved...")
    else:
        for i in range(n):
            img=bundle[f'img_{i}']
            if i%z==0:
                cv2.imwrite(f"{directory}/{i//z:0>3}.png",cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if i==n//2:
                print("...50% saved...")
    print("100% saved!")
    
# save depth or confidence image
# confidence image will be saved as 0,1,2 -> 0,100,200
def save_bundle_x(bundle,x,directory,z=1):
    n=(len(bundle)-1)//4
    check={'depth':'tiff','conf':'png'}
    if x not in check:
        print("No format like",x)
        return
    if not os.path.exists(directory):
            os.makedirs(directory)
    for i in range(n):
        dimg=bundle[f'{x}_{i}'] if x!='conf' else bundle[f'{x}_{i}']*100
        if i%z==0:
            cv2.imwrite(f"{directory}/{i//z:0>3}.{check[x]}",dimg)
        if i==n//2:
            print("...50% saved...")
    print("100% saved!")

In [8]:
def save_ptlist(pt_list,filename):
    pcd_o3d = o3d.geometry.PointCloud()
    points=np.asarray(pt_list[0].points)
    colors=np.asarray(pt_list[0].colors)
    for pcd in pt_list[1:]:
        points=np.concatenate((points,pcd.points),axis=0)
        colors=np.concatenate((colors,pcd.colors),axis=0)
    pcd_o3d.points = o3d.utility.Vector3dVector(points)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors)

    o3d.io.write_point_cloud(f'{filename}.ply', pcd_o3d, write_ascii=False, compressed=False, print_progress=False)
    print(f'save as {filename}.ply')

아래는 ICP 로 이어붙여본 코드입니다. 어떤 confidence 이상의 포인트의 개수가 최대/최소인 프레임의 인덱스 번호를 출력합니다. 이때 최소한 몇 개 이상이어야 하는지 제한을 걸어둘 수 있게끔 했습니다.

In [9]:
import time

def get_points_number(bundle,frame,allowed_conf=2,h=1):
    conf_image=bundle[f'conf_{frame}']
    n=0
    length=len(conf_image)//h 
    for row in conf_image[:length]:
        for e in row:
            if e>=allowed_conf:
                n+=1
    return n

def get_frames(bundle,plim,allowed_conf=2,h=1):
    start=time.time()
    n=get_bundle_length(bundle)
    m=192*256
    mi=0
    M=0
    Mi=0
    idx=[]
    for i in range(n):
        t=get_points_number(bundle,i,allowed_conf,h=h)
        if t>plim:
            idx.append(i)
            if m>t:
                m=t
                mi=i
            if M<t:
                M=t
                Mi=i
    print(f"Get {len(idx)} frames, proceed time: {time.time()-start}")
    return m,mi,Mi,idx


## point3D, images로부터 저장

In [23]:
import os
target='buds'
path = f"data_processed_{target}"
file_list = os.listdir(path)
directory=f'data/{target}'

bundle_list=[]
info_list=[]
img_list=[]
depth_list=[]
conf_list=[]
f_list=[0,1]
for i,filename in enumerate(file_list):
    bundle=np.load(f"{path}/{filename}/frame_bundle.npz", allow_pickle=True)
    info_list.append(bundle[f'info_{f_list[0]}'])
    info_list.append(bundle[f'info_{f_list[1]}'])
    img_list.append(bundle[f'img_{f_list[0]}'])
    img_list.append(bundle[f'img_{f_list[1]}'])
    conf_list.append(bundle[f'conf_{f_list[0]}'])
    conf_list.append(bundle[f'conf_{f_list[1]}'])
    depth_list.append(bundle[f'depth_{f_list[0]}'])
    depth_list.append(bundle[f'depth_{f_list[1]}'])
    bundle_list.append(bundle)
#     save_bundle_imgs(bundle,f"{directory}/bundle_{target}/{filename}",index_list=[0,1])

In [24]:
real_img_num=len(img_list)

intrinsics=[]
for i,info in enumerate(info_list):
    M=info.item()['intrinsics']
    M[1][1],M[0][0]=M[0][0],M[1][1]
    M[1][2],M[0][2]=M[0][2],M[1][2]
    intrinsics.append(M)

real_img_num

24

In [25]:
#images
def save_images(directory):
    image_path=f"{directory}/images"
    if not os.path.exists(image_path):
        os.makedirs(image_path)
    for i,img in enumerate(img_list):
        cv2.imwrite(f"{image_path}/{i:0>3}.png",cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

save_images(directory)

In [26]:
'''
given 
1. poses: list of [T_cw ...]
2. img_names: list of [cam1/001.jpg ...]
3. cam_ids : list of [1 ...]
* len(cam_ids) == len(img_names)
'''
import numpy as np
from scipy.spatial.transform import Rotation as Rot

    
def read_images_txt(images_path,save=False):
    if not os.path.exists(images_path):
        raise Exception(f"No such file : {images_path}")

    with open(images_path, 'r') as f:
        lines = f.readlines()

    if len(lines) < 2:
        raise Exception(f"Invalid cameras.txt file : {images_path}")

    comments = lines[:4]
    contents = lines[4:]

    img_ids = []
    cam_ids = []
    img_names = []
    poses = []
    points2d = {}
    for img_idx, content in enumerate(contents):
        if img_idx%2==0:
            content_items = content.split(' ')
            img_id = content_items[0]
            q_xyzw = np.array(content_items[2:5] + content_items[1:2], dtype=np.float32) # colmap uses wxyz
            t_xyz = np.array(content_items[5:8], dtype=np.float32)
            cam_id = content_items[8]
            img_name = content_items[9]

            R = Rot.from_quat(q_xyzw).as_matrix()
            T = np.eye(4)
            T[:3, :3] = R
            T[:3, -1] = t_xyz

            img_ids.append(img_id)
            cam_ids.append(cam_id)
            img_names.append(img_name)
            poses.append(T)
        else:
#             pass
            n = list(map(float, content.split()))
            new_list = []
            for j in range(0, len(n), 3):
                # if n[j+2]==-1:
                #     continue
                new_list.append(np.array(n[j:j + 3]))
            points2d[(img_idx // 2) + 1] = np.array(new_list)

    if save:
        np.save(f'{images_path}/point2s.npy', points2d)
    return img_ids, cam_ids, img_names, poses, points2d

def read_points_txt(points_path,save=False):
    if not os.path.exists(points_path):
        raise Exception(f"No such file : {points_path}")

    with open(points_path, 'r') as f:
        lines = f.readlines()

    comments = lines[:3] #   POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
    contents = lines[3:]

    points={}
    for content in contents:
        pt3d,x,y,z,r, g, b,err,*track=map(float,content.split())
        points[pt3d] = {'xyz': [x, y, z], 'rgb': [r, g, b], 'err': err, 'track':track}
    
    if save:
        np.save(f'{points_path}/point3s.npy', points)
    return points

In [27]:
for k,i in enumerate(intrinsics):
    if k%2==0:
        print(f"{i[1][1]},{i[0][0]},{i[0][2]},{i[1][2]},0,0,0,0")
    

1609.6375732421875,1609.6375732421875,715.1256103515625,952.4200439453125,0,0,0,0
1627.7137451171875,1627.7137451171875,715.0498657226562,952.4653930664062,0,0,0,0
1602.8590087890625,1602.8590087890625,715.0068969726562,952.5029296875,0,0,0,0
1599.2437744140625,1599.2437744140625,715.2296142578125,952.5748901367188,0,0,0,0
1616.4161376953125,1616.4161376953125,715.167724609375,952.6710205078125,0,0,0,0
1601.5032958984375,1601.5032958984375,715.2330322265625,952.54931640625,0,0,0,0
1604.6666259765625,1604.6666259765625,715.5746459960938,952.7606811523438,0,0,0,0
1591.5614013671875,1591.5614013671875,715.0184936523438,953.5023803710938,0,0,0,0
1611.8970947265625,1611.8970947265625,714.9447021484375,952.5502319335938,0,0,0,0
1602.4071044921875,1602.4071044921875,714.9979248046875,952.5606079101562,0,0,0,0
1598.7918701171875,1598.7918701171875,715.0339965820312,952.6919555664062,0,0,0,0
1607.8299560546875,1607.8299560546875,715.117431640625,952.8091430664062,0,0,0,0


In [28]:
img_ids, cam_ids, img_names, w2c_poses,pt2d=read_images_txt(f"{directory}/images.txt")
pt3d=read_points_txt(f"{directory}/points3D.txt")

# pt2d=np.load(f'{path}/point2s.npy', allow_pickle=True).item() #[x,y pt3d] array per image || key= 1, 2, ... img_num
# pt3d=np.load(f'{path}/point3s.npy', allow_pickle=True).item() #xyz,rgb,err for all pt3d image

img_num=len(pt2d)
print(len(pt2d),len(pt3d))
print(len(w2c_poses))

24 15403
24


In [29]:
print(M@[0,0,1])

[715.11804199 952.80834961   1.        ]


In [15]:
def euclidistance(p1,p2):
    dist=(p2[0]-p1[0])**2
    assert len(p1)==len(p2)
    for i in range(1,len(p1)):
        dist+=(p2[i]-p1[i])**2
    return np.sqrt(dist)

In [30]:
#for 1440,1920
import math

def get_scale_factor(idx,pt2d,col_pt3d):
    M=intrinsics[idx]
    K=w2c_poses[idx]
    v4=[*col_pt3d,1]
    
    projection=K@v4
    projection=M@projection[:3]
    
    zc=projection[2]
    c,r=pt2d #np.round(np.array(pt2d)/7.5)
    print(r,c,end=' ')
#     r= int(r) if r>=255.9 else 255
#     c=int(c) if c>=191.9 else 191
    print(r,c)
#     r=round(r/7.5)
    depth=depth_list[idx][math.floor(r/7.5)][math.floor(c/7.5)]
#     depth=depth_list[idx][round(r/7.5)][round(c/7.5)]
#     depth=depth_list[idx][r][c]
#     depth=depth_list[idx][round(r)//7.5][round(c)//7.5]
    
    factor=zc/depth
#     print(factor)
    return factor

scales=[]
print(pt2d[1])
for i in range(img_num):
    start=time.time()
    fac=[]
    for x,y,id3 in pt2d[i+1]:
        if id3==-1:
            continue
        print(i,[x,y])
        factor=get_scale_factor(i,[x,y],pt3d[id3]['xyz'])
        fac.append(factor)
    mean_scale=np.sum(fac)/len(fac)
    print("------",i,">",mean_scale)
    print("time:",time.time()-start)
    scales.append(mean_scale)
scales

[[ 8.01180573e+01  2.15688419e+01 -1.00000000e+00]
 [ 1.50778534e+02  5.73512602e+00 -1.00000000e+00]
 [ 3.48455902e+02  4.35226917e+00 -1.00000000e+00]
 ...
 [ 8.65921509e+02  4.68014404e+02 -1.00000000e+00]
 [ 1.04796021e+03  9.68792969e+02 -1.00000000e+00]
 [ 3.52654114e+02  5.19647339e+02 -1.00000000e+00]]
0 [377.2126159667969, 32.654117584228516]
32.654117584228516 377.2126159667969 32.654117584228516 377.2126159667969
0 [460.186767578125, 77.90460205078125]
77.90460205078125 460.186767578125 77.90460205078125 460.186767578125
0 [560.4439697265625, 144.7161865234375]
144.7161865234375 560.4439697265625 144.7161865234375 560.4439697265625
0 [572.9100952148438, 162.109130859375]
162.109130859375 572.9100952148438 162.109130859375 572.9100952148438
0 [665.9022216796875, 167.49241638183594]
167.49241638183594 665.9022216796875 167.49241638183594 665.9022216796875
0 [665.9022216796875, 167.49241638183594]
167.49241638183594 665.9022216796875 167.49241638183594 665.9022216796875
0 [226.

3 [892.2861938476562, 1119.8653564453125]
1119.8653564453125 892.2861938476562 1119.8653564453125 892.2861938476562
3 [984.8934936523438, 1094.2979736328125]
1094.2979736328125 984.8934936523438 1094.2979736328125 984.8934936523438
3 [1030.533203125, 1108.0050048828125]
1108.0050048828125 1030.533203125 1108.0050048828125 1030.533203125
3 [1183.826416015625, 1091.484130859375]
1091.484130859375 1183.826416015625 1091.484130859375 1183.826416015625
3 [1247.3905029296875, 1106.3193359375]
1106.3193359375 1247.3905029296875 1106.3193359375 1247.3905029296875
3 [29.026081085205078, 1136.667724609375]
1136.667724609375 29.026081085205078 1136.667724609375 29.026081085205078
3 [60.21253204345703, 1141.0701904296875]
1141.0701904296875 60.21253204345703 1141.0701904296875 60.21253204345703
3 [60.21253204345703, 1141.0701904296875]
1141.0701904296875 60.21253204345703 1141.0701904296875 60.21253204345703
3 [73.72634887695312, 1123.749755859375]
1123.749755859375 73.72634887695312 1123.74975585

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



------ 23 > 19.238620032502414
time: 0.13169598579406738


[28.31753377914052,
 28.281456775764116,
 17.602644402356514,
 17.582298919199943,
 18.165469265918365,
 18.170195270795862,
 18.59325733241156,
 18.557809518459834,
 21.714864358123382,
 21.741474034819674,
 18.732095443064654,
 18.737488201714427,
 19.701180262955003,
 19.684671180663113,
 14.100384796920949,
 14.16010565248426,
 19.254788148519992,
 19.255248452575632,
 19.23177362767594,
 19.239917738985483,
 19.246360118078883,
 19.250260063380864,
 19.24354892268148,
 19.238620032502414]

In [17]:
def get_pt_w(idx,allowed_conf=2,color_offset=[0,0,0],factor=None):
    rgb_image=img_list[idx]
    depth_image=cv2.resize(depth_list[idx],dsize=(1440,1920),interpolation=cv2.INTER_LINEAR)
    conf_image=cv2.resize(conf_list[idx],dsize=(1440,1920),interpolation=cv2.INTER_LINEAR)
    w2c=w2c_poses[idx]
    M=intrinsics[idx]
    fx,fy,cx,cy=M[0][0],M[1][1],M[0][2],M[1][2]
    pcd,colors = [],[]
    
    R=np.linalg.inv(w2c[:3,:3])
    t=w2c[:3,3]
    height, width = depth_image.shape
    for i in range(height):
        for j in range(width):
            if conf_image[i][j]<allowed_conf:
                continue
            #normalize coordinate
            z = depth_image[i][j]
            if factor:
                z*=factor
            x = (j - cx) * z / fx 
            y = (i - cy) * z / fy
            vec= R@([x,y,z]-t)
            pcd.append(vec)
            colors.append((rgb_image[i][j]+color_offset)/255)
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
    
    return pcd_o3d

def get_pt_w_lidar(idx,allowed_conf=2,color_offset=[0,0,0],factor=None):
    rgb_image=cv2.resize(img_list[idx],dsize=(192,256))
    depth_image=depth_list[idx]
    conf_image=conf_list[idx]
    w2c=w2c_poses[idx]
    M=intrinsics[idx]/7.5
    fx,fy,cx,cy=M[0][0],M[1][1],M[0][2],M[1][2]
    pcd,colors = [],[]
    
    R=np.linalg.inv(w2c[:3,:3])
    t=w2c[:3,3]
    height, width = depth_image.shape
    for i in range(height):
        for j in range(width):
            if conf_image[i][j]<allowed_conf:
                continue
            #normalize coordinate
            z = depth_image[i][j]
            if factor:
                z*=factor
            x = (j - cx) * z / fx 
            y = (i - cy) * z / fy
            vec= R@([x,y,z]-t)
            pcd.append(vec)
            colors.append((rgb_image[i][j]+color_offset)/255)
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
    
    return pcd_o3d


In [18]:
M

array([[1.5987919e+03, 0.0000000e+00, 7.1519977e+02],
       [0.0000000e+00, 1.5987919e+03, 9.5408545e+02],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [19]:
def get_info_w_lidar(idx,allowed_conf=2,factor=None):
    rgb_image=cv2.resize(img_list[idx],dsize=(192,256))
    depth_image=depth_list[idx]
    conf_image=conf_list[idx]
    w2c=w2c_poses[idx]
    M=intrinsics[idx]/7.5
    fx,fy,cx,cy=M[0][0],M[1][1],M[0][2],M[1][2]
    pcd,colors,errors,xy2d = [],[],[],[]
    
    R=np.linalg.inv(w2c[:3,:3])
    t=w2c[:3,3]
    height, width = depth_image.shape
    print
    for i in range(height):
        for j in range(width):
            if conf_image[i][j]<allowed_conf:
                continue
            #normalize coordinate
            z = depth_image[i][j]
            if factor:
                z*=factor
            x = (j - cx) * z / fx 
            y = (i - cy) * z / fy
            vec= R@([x,y,z]-t)
            test=w2c@[*vec,1]
            test=intrinsics[idx]@test[:3]
            test_p=test/test[2]
            px=j*7.5 if j%2==0 else j*7.5-.05
            py=i*7.5 if i%2==0 else i*7.5-.05
            dist=euclidistance(test_p[:2],[px,py])
#             print(vec, test_p,[j*7.5,i*7.5],"=>",dist)
            pcd.append(vec)
            colors.append(rgb_image[i][j]/255)
            errors.append(dist)
            xy2d.append([px,py])
    
    return pcd,colors,errors,xy2d

# i=0
# test=get_pt_w_lidar_err(i,factor=scales[i])

In [31]:
def count_arr2d_x(arr,x):
    cnt=0
    h,w=arr.shape
    for i in range(h):
        for j in range(w):
            if arr[i][j] == x:
                cnt+=1
    return cnt
def count_arr2d_xs(arr,x):
    x= set(x)
    cnt=0
    h,w=arr.shape
    for i in range(h):
        for j in range(w):
            if arr[i][j] in x:
                cnt+=1
    return cnt

test=[]
for con in conf_list:
    test.append(count_arr2d_x(con,2))

img_index=sorted(range(img_num),key=lambda x:-test[x])

print(img_index)


[14, 17, 15, 16, 2, 23, 13, 22, 12, 6, 20, 7, 21, 9, 3, 8, 0, 18, 11, 1, 19, 5, 4, 10]


In [ ]:
checked=set()
temp=[]
new_img_index=[]
for i in img_index:
    k=i//2
    if k not in checked:
        checked.add(k)
        new_img_index.append(i)
    else:
        temp.append(i)
new_img_index+=temp
new_img_index

In [ ]:
def projection(pt,im_id):
    x,y,z=pt
    M=intrinsics[im_id]
    K=w2c_poses[im_id]
    result=K@[x,y,z,1]
    result=M@result[:3]
    return np.round(result/result[2], 3)

print(projection([-1.9045687421396442, -4.1331994950879496, 7.0501730809136705],3))
pt2d[4][616]

In [ ]:
def check_all_indices(li,inum):
    num_odd=0
    num_even=0
    for im_id in li[:inum]:
        if im_id%2==0:
            num_even+=1
        else:
            num_odd+=1
    print(inum,num_odd,num_even)
    if not num_even:
        for i in li[inum:]:
            if i%2==0:
                return li[:inum]+[i]
    elif not num_odd:
        for i in li[inum:]:
            if i%2!=0:
                print(i,"add more odd")
                return li[:inum]+[i]
    else:
        return li[:inum]

In [ ]:
def save_lidar_1pt(directory):
    
    img2d=[{} for _ in range(img_num)]
    pc3d_i=[]
    npt_list=[]
    pi=0
    
    for im_id in range(img_num):
        start=time.time()
        pcd,colors,errors,xy2d=get_info_w_lidar(im_id,factor=scales[im_id])
        pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
        pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
        pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
        npt_list.append(pcd_o3d)
        for k,pt in enumerate(pcd):
            img2d[im_id][tuple(xy2d[k])]=[errors[k],pi]
            nst={im_id}
            pi+=1
            pc3d_i.append({'xyz':pt, 'cams':nst})
        print(im_id,pi,time.time()-start)
    
    np.save(f"{directory}/lidar_im3d_1",pc3d_i)    
    np.save(f"{directory}/lidar_im2d_1",img2d)

In [ ]:
save_lidar_1pt(directory)

In [ ]:
img2d=[{} for _ in range(img_num)]
pc3d_i=[]
npt_list=[]
pi=0
indexes=check_all_indices(new_img_index,real_img_num//4)
# indexes=range(img_num)
print(indexes)
for im_id in indexes:
    start=time.time()
    type0 = im_id%2==0
#     if type0:
#         check_list=[i for i in range(0,img_num,2) if i!=im_id]
#         check_list.append(im_id+1)
#     else:
#         check_list=[i for i in range(1,img_num,2) if i!=im_id]
#         check_list.append(im_id-1)
    check_list=range(img_num)
        
    pcd,colors,errors,xy2d=get_info_w_lidar(im_id,factor=scales[im_id])
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
    npt_list.append(pcd_o3d)
    for k,pt in enumerate(pcd):
        img2d[im_id][tuple(xy2d[k])]=[errors[k],pi]
        nst={im_id}
        err=errors[k]
        for i in check_list:
            x,y,_=projection(pt,i)
            if x<0 or y<0 or x>=192 or y>=256:
                continue
            d=depth_list[i][int(y/7.5)][int(x/7.5)]*scales[i]
            c=-w2c_poses[i][:3,:3].T@w2c_poses[i][:3,3]
            d2=euclidistance(pt,c)
            if (x,y) in img2d[i]:
                if img2d[i][(x,y)][0]>abs(d2-d):
                    print("**this case")
                    img2d[i][(x,y)]=[(img2d[im_id][(x,y)][0]*.2+abs(d2-d))/scales[i],pi]
                    pcd3d_i[img2d[im_id][(x,y)][1]]['cams'].remove(i)
                    nst.add(i)
            else:
                img2d[i][(x,y)]=[abs(d2-d)/scales[i],pi] 
                nst.add(i)
        pi+=1
        pc3d_i.append({'xyz':pt, 'cams':nst, 'err':err})
    print(im_id,pi,time.time()-start)
    
# np.save(f"{directory}/lidar_im3d",pc3d_i)

In [32]:
scales

[28.31753377914052,
 28.281456775764116,
 17.602644402356514,
 17.582298919199943,
 18.165469265918365,
 18.170195270795862,
 18.59325733241156,
 18.557809518459834,
 21.714864358123382,
 21.741474034819674,
 18.732095443064654,
 18.737488201714427,
 19.701180262955003,
 19.684671180663113,
 14.100384796920949,
 14.16010565248426,
 19.254788148519992,
 19.255248452575632,
 19.23177362767594,
 19.239917738985483,
 19.246360118078883,
 19.250260063380864,
 19.24354892268148,
 19.238620032502414]

In [33]:
img2d=[{} for _ in range(img_num)]
pc3d_i=[]
pt_list=[]
pi=0
# indexes=check_all_indices(new_img_index,real_img_num//4)
indexes=range(img_num)
print(indexes)
for im_id in indexes:
    start=time.time()
            
    pcd,colors,errors,xy2d=get_info_w_lidar(im_id,factor=scales[im_id])
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
    pt_list.append(pcd_o3d)
    
    print(im_id,pi,time.time()-start)
    
# np.save(f"{directory}/lidar_im3d",pc3d_i)

range(0, 24)
0 0 1.4148902893066406
1 0 1.327087163925171
2 0 1.3842709064483643
3 0 1.3221616744995117
4 0 1.3954460620880127
5 0 1.3217356204986572
6 0 1.3780739307403564
7 0 1.319725513458252
8 0 1.3801076412200928
9 0 1.3232343196868896
10 0 1.374356985092163
11 0 1.3042778968811035
12 0 1.3978281021118164
13 0 1.3207526206970215
14 0 1.3852593898773193
15 0 1.3238818645477295
16 0 1.4083235263824463
17 0 1.3145546913146973
18 0 1.3665354251861572
19 0 1.3197107315063477
20 0 1.379065752029419
21 0 1.3832833766937256
22 0 1.321732997894287
23 0 1.3957960605621338


In [37]:
o3d.visualization.draw_geometries(pt_list[2:15])

In [ ]:
for pt in pt_list:
    o3d.visualization.draw_geometries([pt])

In [ ]:
for p3 in pc3d_i:
    for c in p3['cams']:
        x,y,_=projection(p3['xyz'],c)
        img2d[c][(x,y)]=p3['err']

In [ ]:
print(img2d[0])

In [ ]:
pc3d_i[56]

In [ ]:
projection(pc3d_i[56]['xyz'],1), np.array(img2d[0][(5.618, 89.415)])/scales[0],pc3d_i[56]['err']

In [ ]:
def projection_err(pt,cam_list,depth):
    x,y,z=pt
    err=0
    for c in cam_list:
        M=intrinsics[c]
        K=w2c_poses[c]
        result=K@[x,y,z,1]
        result=M@result[:3]
        result=result/result[2]
        dist=euclidistance(result[:2],[x,y])
        img2d[i][(x,y)]=[dist] 
    for i in id_list:

In [ ]:
pc3d_i[0]

In [ ]:
cnt=0
for p in pc3d_i:
    xyz=p['xyz']
    cam=p['cams']
    if len(cam)>5:
        print(xyz,cam)
        cnt+=1
cnt

In [ ]:
indexes=[]
for i,img in enumerate(img2d):
    if len(img.keys())==0:
        indexes.append(i)
    print(len(img.keys()))
    
print(indexes)
    
    
for im_id in indexes:
    start=time.time()
    pcd,colors,errors,xy2d=get_info_w_lidar(im_id,factor=scales[im_id])
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors) #set colors
    npt_list.append(pcd_o3d)
    for k,pt in enumerate(pcd):
        img2d[im_id][tuple(xy2d[k])]=[errors[k],pi]
        nst={im_id}
        pi+=1
        pc3d_i.append({'xyz':pt, 'cams':nst})
    print(im_id,pi,time.time()-start)

for i,img in enumerate(img2d):
    print(len(img.keys()))


In [ ]:
np.save(f"{directory}/lidar_im3d",pc3d_i)    
np.save(f"{directory}/lidar_im2d",img2d)
save_lidar_1pt(directory)

In [ ]:
pi, npt_list

In [ ]:
np.sum(npt_list)

In [ ]:
# save_ptlist(ipt_list,"recon_top_4")
save_ptlist(npt_list,f"{directory}/recon_topd_4")

In [ ]:

pcd = o3d.io.read_point_cloud(f"{directory}/recon_topd_4.ply")
# np.save("lidar_im3d",pcd.points)

In [ ]:
lp3=np.load(f"{directory}/lidar_im3d.npy",allow_pickle=True)
lp2=np.load(f"{directory}/lidar_im2d.npy",allow_pickle=True)
lp3

In [ ]:
len(lp2[0].keys())
keys=list(lp2[0].keys())
id3=lp2[0][keys[5]]
# print(lp2[0])
print(id3)
print(lp3[id3[1]])

In [ ]:
img_id=1
test=list(lp2[img_id].items())
print(test[:4])

In [ ]:
np.array(list(lp2[0].values()))[:,1]

In [ ]:
o3d.visualization.draw_geometries(ipt_list)

In [ ]:
o3d.visualization.draw_geometries(npt_list)

In [ ]:
print(img_index)

In [ ]:
lidar_info=[]
for i in range(img_num):
    lidar={}
    pcd,colors,errors,xy2d=get_info_w_lidar(i,factor=scales[i])
    lidar['xyz']=pcd
    lidar['rgb']=colors
    lidar['error']=errors
    lidar['img2d']=xy2d
    lidar_info.append(lidar)
lidar_info

In [ ]:
np.save(f'{directory}/lidar_info',lidar_info)

In [ ]:
print(scales)

In [ ]:
min_depth=np.min(depth_list)
min_d_index=np.argmin(depth_list)
mean_scale=np.mean(scales)
max_depth=np.max(depth_list)
print(min_depth,mean_scale,min_d_index)
print(min_depth*mean_scale)
print(max_depth*mean_scale, max_depth)

In [ ]:
a=np.array([1,1,1]).reshape(-1,3)
b=np.array([2,2,2]).reshape(-1,3)
a=np.array([1,1,1])
b=np.array([2,2,2])
c=np.array([])
np.concatenate((a,b))

In [ ]:
sorted_pt3d_keys=sorted(pt3d.keys(),key=lambda x:pt3d[x]['err'])

In [ ]:
checked=set(range(1,25))
for i in sorted_pt3d_keys[:5]:
    print(i,pt3d[i])
    track=pt3d[i]["track"]
    for t,e in enumerate(track):
        if t%2==0:
            cam_id=e
        else:
            img_id=e
            x,y,p=pt3d[cam_id][int(img_id)]
            assert p==i
            x,y=x/7.5,y/7.5
            rx,ry=round(x),round(y)
            bundle=bundle_list[cam_id//2]
            frame=cam_id%2
            conf=bundle[f'conf_{frame}'][ry][rx]
            if not conf:
                continue
            depth=bundle[f'depth_{frame}'][ry][rx]
            M=bundle[f'info_{frame}'].item()['intrinsics']/7.5
            fx=M[0][0]
            fy=M[1][1]
            cx=M[0][2]
            cy=M[1][2]
            x = (j - cx) / fx 
            y = (i - cy) / fy
            R=c2ws[cam_id][:3,:3]
            t=c2ws[cam_id][:3,3]
            n2w=R@[x,y,1]+t
            dist=euclidistance(n2w[:3],pt3d[cam_id]['xyz'])
            print()

In [ ]:
checked=set(range(1,25))
for i in sorted_pt3d_keys[:5]:
    print(i,pt3d[i])
    track=pt3d[i]['track']
    for t,e in enumerate(track):
        if t%2==0:
            img_id=int(e)
        else:
            img_idx=int(e)
            print(">",img_id,img_idx)
            x,y,p=pt2d[img_id][img_idx]
            assert p==i
            x,y=x/7.5,y/7.5
            rx,ry=round(x),round(y)
            bundle=bundle_list[(img_id-1)//2]
            frame=img_id%2
            conf=bundle[f'conf_{frame}'][ry][rx]
            if not conf:
                continue
            depth=bundle[f'depth_{frame}'][ry][rx]
            M=bundle[f'info_{frame}'].item()['intrinsics']/7.5
            fx=M[0][0]
            fy=M[1][1]
            cx=M[0][2]
            cy=M[1][2]
            x = (x - cx) / fx 
            y = (y - cy) / fy
            w2c=w2c_poses[img_id-1] #np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])@
            R=w2c[:3,:3]
            t=w2c[:3,3]
            cam3d=R@pt3d[img_id-1]['xyz']+t
            a=euclidistance([0,0,0],[x,y,1])
            b=euclidistance([0,0,0],cam3d)
            fac=b/(a*depth)
            print(fac,":",img_id,"with conf",conf)

In [ ]:
pt_list[4-1]=bundle2pt_trans(bundle_list[1],1,c2ws[3], factor=11,color_offset=[30,0,0])
pt_list[15-1]=bundle2pt_trans(bundle_list[7],0,c2ws[14], factor=13,color_offset=[30,0,0])
pt_list[16-1]=bundle2pt_trans(bundle_list[7],1,c2ws[15], factor=13,color_offset=[30,0,0])


o3d.visualization.draw_geometries(pt_list)
# for i in range(12):
#     pt_list.append(bundle2pt_trans(bundle_list[i],0,np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])@c2ws[i*2], factor=scales[2*i]))
#     pt_list.append(bundle2pt_trans(bundle_list[i],1,np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])@c2ws[i*2+1],factor=scales[2*i+1]))
#     print("done",i)

### 시행착오들...